In [1]:
import sklearn as sk
import pandas as pd
import numpy as np
import urllib
import scipy.optimize
import random
import gzip
from collections import defaultdict
import warnings
import statistics
warnings.filterwarnings('ignore')

In [2]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N
#store the answer
answers = {}
#read the data from file'5year.arff'
f = open("data/5year.arff", 'r')
# Read and parse the data
while not '@data' in f.readline():
    pass

dataset = []
for l in f:
    if '?' in l: # Missing entry
        continue
    l = l.split(',')
    values = [1] + [float(x) for x in l]
    values[-1] = values[-1] > 0 # Convert to bool
    dataset.append(values)

In [3]:
#Task1----------------------------------------------------------------------------
#build logistic model
datasetNP=np.array(dataset)
dataX=datasetNP[:,:-1]
dataY=datasetNP[:,-1]
from sklearn.linear_model import LogisticRegression
model_Task1=LogisticRegression(C=1.0)
model_Task1.fit(dataX,dataY)
#calculate the accuracy and Balanced Error Rate (BER)
def accuracy_BER_calculation(y,y_predict):
    TruePt,TrueNg,FalsePt,FalseNg=0,0,0,0
    for i in range(len(y_predict)):
        if y_predict[i]==0:
            if y[i]==0:
                TrueNg+=1
            else:
                FalseNg+=1
        else:
            if y[i]==1:
                TruePt+=1
            else:
                FalsePt+=1
    BER=0.5*((FalsePt/(FalsePt+TrueNg))+(FalseNg/(FalseNg+TruePt)))
    accuracy=(TrueNg+TruePt)/len(y)
    return [accuracy,BER]
dataY_pre1=model_Task1.predict(dataX)
answers['Q1']= accuracy_BER_calculation(dataY,dataY_pre1)
assertFloatList(answers['Q1'], 2)
print(answers['Q1'])

[0.9656878917848895, 0.47668514315934635]


In [4]:
#Task2----------------------------------------------------------------------------
model_Task2=LogisticRegression(C=1.0,class_weight='balanced')
model_Task2.fit(dataX,dataY)
dataY_pre2=model_Task2.predict(dataX)
answers['Q2']= accuracy_BER_calculation(dataY,dataY_pre2)
assertFloatList(answers['Q2'], 2)
print(answers['Q2'])

[0.6961398878258, 0.30388977031577397]


In [5]:
#Task3----------------------------------------------------------------------------
random.seed(3)
random.shuffle(dataset)
datasetNP3=np.array(dataset)
dataX3=datasetNP3[:,:-1]
dataY3=datasetNP3[:,-1]
length=len(dataX3)
Xtrain, Xvalid, Xtest = dataX3[:length//2], dataX3[length//2:(3*length)//4], dataX3[(3*length)//4:]
ytrain, yvalid, ytest = dataY3[:length//2], dataY3[length//2:(3*length)//4], dataY3[(3*length)//4:]
model_Task3=LogisticRegression(C=1.0,class_weight='balanced')
model_Task3.fit(Xtrain,ytrain)
ytrain_pre=model_Task3.predict(Xtrain)
yvalid_pre=model_Task3.predict(Xvalid)
ytest_pre=model_Task3.predict(Xtest)
[berTrain, berValid, berTest]=[accuracy_BER_calculation(ytrain,ytrain_pre)[1],\
                               accuracy_BER_calculation(yvalid,yvalid_pre)[1],\
                               accuracy_BER_calculation(ytest,ytest_pre)[1]]
answers['Q3'] = [berTrain, berValid, berTest]
assertFloatList(answers['Q3'], 3)
print(answers['Q3'])

[0.29287226079549855, 0.3159203980099502, 0.2585616438356164]


In [6]:
#Task4----------------------------------------------------------------------------
C_val=[10**(-4),10**(-3),10**(-2),10**(-1),1,10,10**2,10**3,10**4]
#Report the validation BER
berList=[]
for c in C_val:
    model_Task4=LogisticRegression(C=c,class_weight='balanced')
    model_Task4.fit(Xtrain,ytrain)
    yvalid_pre=model_Task4.predict(Xvalid)
    berList.append(accuracy_BER_calculation(yvalid,yvalid_pre)[1])
answers['Q4'] = berList
assertFloatList(answers['Q4'], 9)
print(berList)

[0.32406151062867483, 0.31931252826775214, 0.3301673450927182, 0.3227046585255541, 0.3159203980099502, 0.3111714156490276, 0.2955030044582283, 0.29618143050978873, 0.29618143050978873]


In [7]:
#Task5----------------------------------------------------------------------------
berList5=[]
bestC=1.0
for c in C_val:
    model_Task5=LogisticRegression(C=c,class_weight='balanced')
    model_Task5.fit(Xtrain,ytrain)
    ytest_pre=model_Task5.predict(Xtest)
    berList5.append(accuracy_BER_calculation(ytest,ytest_pre)[1])
print(berList5)

[0.26335616438356163, 0.25582191780821917, 0.26472602739726026, 0.26404109589041097, 0.2585616438356164, 0.2723091976516634, 0.26267123287671235, 0.26267123287671235, 0.26267123287671235]


In [8]:
bestC=100
ber5=berList5[6]
answers['Q5'] = [bestC, ber5]
assertFloatList(answers['Q5'], 2)
print(answers['Q5'])

[100, 0.26267123287671235]


In [9]:
#Task6----------------------------------------------------------------------------
data_taskRecommend=pd.read_json('data/young_adult_10000.json', lines=True)
dataTrain = data_taskRecommend[:9000]
dataTest = data_taskRecommend[9000:]

In [10]:
data_taskRecommend.iloc[0]

user_id                          8842281e1d1347389f2ab93d60773d4d
book_id                                                   2767052
review_id                        248c011811e945eca861b5c31a549291
rating                                                          5
review_text     I cracked and finally picked this up. Very enj...
date_added                         Wed Jan 13 13:38:25 -0800 2010
date_updated                       Wed Mar 22 11:46:36 -0700 2017
read_at                            Sun Mar 25 00:00:00 -0700 2012
started_at                         Fri Mar 23 00:00:00 -0700 2012
n_votes                                                        24
n_comments                                                     25
Name: 0, dtype: object

In [19]:
usersPerBook = defaultdict(set)
booksPerUser = defaultdict(set)
reviewsPerUser = defaultdict(list)
reviewsPerBook = defaultdict(list)
globalAvg=0
trainRatingDict = {} # To retrieve a rating for a specific user/item pair
for i in range(len(dataTrain)):
    d=dataTrain.iloc[i,:]
    user,book = d['user_id'], d['book_id']
    rating=d['rating']
    globalAvg+=rating
    usersPerBook[book].add(user)
    booksPerUser[user].add(book)
    reviewsPerUser[user].append(rating)
    reviewsPerBook[book].append(rating)
    trainRatingDict[(user,book)]=rating
globalAvg=globalAvg/len(dataTrain)

In [20]:
def JaccardSimilarity(set1,set2):
    inter = len(set1.intersection(set2))
    uni = len(set1.union(set2))
    return inter/uni
def mostKSimilar (targetedBook, K): 
    JSimList = []
    users = usersPerBook[targetedBook] 
    for comparedBook in usersPerBook : 
        if targetedBook == comparedBook: 
            continue
        similarity = JaccardSimilarity(users,usersPerBook[comparedBook])
        JSimList.append((similarity ,comparedBook))
    JSimList.sort(key=lambda x:x[0],reverse=True) # Sort to find the most similar
    return JSimList[:K]
answers['Q6'] = mostKSimilar(2767052, 10)
assert len(answers['Q6']) == 10
assertFloatList([x[0] for x in answers['Q6']], 10)

In [21]:
answers['Q6']

[(0.4125, 6148028),
 (0.3411764705882353, 7260188),
 (0.1590909090909091, 256683),
 (0.1375, 1162543),
 (0.11494252873563218, 11735983),
 (0.10989010989010989, 13335037),
 (0.10810810810810811, 28187),
 (0.10666666666666667, 428263),
 (0.09876543209876543, 49041),
 (0.09782608695652174, 41865)]

In [38]:
#Task7----------------------------------------------------------------------------
avgRatingPerBook=defaultdict(float)
for book in reviewsPerBook:
    ReviewList=reviewsPerBook[book]
    avgRatingPerBook[book]=statistics.mean(ReviewList)
#testset
testRatingDict = {} 
for i in range(len(dataTest)):
    d=dataTest.iloc[i,:]
    user,book = d['user_id'], d['book_id']
    rating=d['rating']
    testRatingDict[(user,book)]=rating

#calculate global average ratings of all books
predictUserBook=[]
sumSim=[]
for (user,book) in testRatingDict:
    numerator=0
    denominator=0
    users= usersPerBook[book] 
    for comparedBook in booksPerUser[user]:
        sim_ij=0
        if book == comparedBook: 
            continue
        sim_ij = JaccardSimilarity(users,usersPerBook[comparedBook])
        denominator+=(sim_ij)
        numerator+=((trainRatingDict[(user,comparedBook)]-avgRatingPerBook[comparedBook])*sim_ij)
    sumSim.append(denominator)
    if  denominator!=0:
        ans=(avgRatingPerBook[book]+(numerator/denominator))  
    else:
        ans= globalAvg
    predictUserBook.append(ans)
#calculate MSE
SquaredError=0
for i,(user,book) in enumerate(testRatingDict):
    Error=testRatingDict[(user,book)]-predictUserBook[i]
    SquaredError+=(Error**2)
mse7=SquaredError/(len(testRatingDict))
answers['Q7'] = mse7
assertFloat(answers['Q7'])
print(mse7)

1.2514617362474678


In [40]:
#Task8----------------------------------------------------------------------------
avgRatingPerUser=defaultdict(float)
for user in reviewsPerUser:
    ReviewList=reviewsPerUser[user]
    avgRatingPerUser[user]=statistics.mean(ReviewList)
predictUserBook=[]
for (user,book) in testRatingDict:
    numerator=0
    denominator=0
    books = booksPerUser[user] 
    for comparedUser in usersPerBook[book]:
        if user == comparedUser: 
            continue
        sim_uv = JaccardSimilarity(books,booksPerUser[comparedUser])
        denominator+=(sim_uv)
        numerator+=((trainRatingDict[(comparedUser,book)]-avgRatingPerUser[comparedUser])*sim_uv)
    if  denominator!=0:
        ans=(avgRatingPerUser[user]+(numerator/denominator))  
    else:
        ans= globalAvg
    predictUserBook.append(ans)
#calculate MSE
SquaredError=0
for i,(user,book) in enumerate(testRatingDict):
    Error=testRatingDict[(user,book)]-predictUserBook[i]
    SquaredError+=Error**2
mse8=SquaredError/(len(testRatingDict))
answers['Q8'] = mse8
assertFloat(answers['Q8'])
print(mse8)
f = open("answers_hw2.txt", 'w')
f.write(str(answers) + '\n')
f.close()

1.2461869156447403
